In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
pd.set_option('display.max_columns', None)

In [2]:
dataPath = "data/"

pairs = pd.read_csv(dataPath + "relation.csv")
users = pd.read_csv(dataPath + "customers.csv", index_col="id")
items = pd.read_csv(dataPath + "products.csv", index_col="id")


/tmp/ipykernel_776/1774659839.py:4: DtypeWarning: Columns (7,9,10,11,12,13,17,18,21,26,27,28,30,36) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv(dataPath + "customers.csv", index_col="id")
/tmp/ipykernel_776/1774659839.py:5: DtypeWarning: Columns (20,32,34,39) have mixed types. Specify dtype option on import or set low_memory=False.
  items = pd.read_csv(dataPath + "products.csv", index_col="id")


### Compañia 15

In [3]:
df_pairs = pairs[ pairs.company_id == 15]#.copy()
df_users = users[ users.company_id == 15]#.copy()
df_items = items[ items.company_id == 15]#.copy()

In [4]:
#74795,  29133,  28903
#df_items.loc[28903]

In [5]:
# Comprobacion de que no haya usuarios ni productos duplicados

assert df_users.index.duplicated().any()==False
assert df_items.index.duplicated().any()==False

### Quitar usuarios que interaccionan pero no estan contemplados en la tabla usuarios (unos 575)

In [6]:
usuarios_que_interaccionan_pero_no_en_tabla_usuarios = set(df_pairs.customer_id) - set(df_users.index)
len(usuarios_que_interaccionan_pero_no_en_tabla_usuarios)

575

In [7]:
df_pairs = df_pairs[ ~df_pairs.customer_id.isin(usuarios_que_interaccionan_pero_no_en_tabla_usuarios) ]
len(df_pairs)

59167

### Quitar productos que interaccionan pero no estan contemplados en la tabla productos (unos 162)

In [8]:
productos_que_interaccionan_pero_no_en_tabla_productos = set(df_pairs.product_id) - set(df_items.index)
len(productos_que_interaccionan_pero_no_en_tabla_productos)

162

In [9]:
df_pairs = df_pairs[ ~df_pairs.product_id.isin(productos_que_interaccionan_pero_no_en_tabla_productos) ]
len(df_pairs)

58913

# <center> Fin código limepieza datos
---
# <center> Inicio código Sistema de recomendación

In [10]:
from recommenderSystem2 import RecSys2
r = RecSys2()

In [11]:
r.entrenar_modelo(df_pairs, df_users, df_items)

Leyendo datos...
Entrenando modelo...


100%|█████████████████████████████████████████| 921/921 [00:16<00:00, 54.77it/s]


tensor(3.6570, grad_fn=<MseLossBackward0>)


100%|█████████████████████████████████████████| 921/921 [00:16<00:00, 56.74it/s]

tensor(0.8313, grad_fn=<MseLossBackward0>)
Generando embeddings...
Terminado


In [12]:
r.activeUser_ids[:5], r.newUser_ids[:5]

(tensor([21724, 21726, 21728, 21730, 21731]),
 tensor([21725, 21727, 21729, 21742, 21744]))

In [13]:
r.activeItem_ids[:5], r.newItem_ids[:5]

(tensor([28863, 28864, 28865, 28866, 28867]),
 tensor([28871, 28873, 28879, 28881, 28936]))

In [ ]:
r.recomendar(query_user_ids=[21724, 21726], query_item_ids=[28863, 28864, 28865], peso_query_user=0.3,
            searchOn_act_users=True, limit=0.9)

In [14]:
# alfombra para uselo
r.recomendar(query_item_ids=[29021], searchOn_act_items=True, limit=0.6)[:10]

[(29021, 1.0),
 (36419, 0.8908638954162598),
 (28907, 0.7940273284912109),
 (111010, 0.7760097980499268),
 (29352, 0.7749708294868469),
 (29071, 0.7201521396636963),
 (39207, 0.7195676565170288),
 (29347, 0.7153081893920898),
 (113230, 0.6916004419326782),
 (29061, 0.6858459711074829)]

In [17]:
# raton
r.recomendar(query_item_ids=[28891], searchOn_act_items=True, limit=0.6)[:10]

[(28891, 1.0),
 (29195, 0.8482922315597534),
 (39195, 0.7612780928611755),
 (74818, 0.7444325685501099),
 (126436, 0.725811779499054),
 (29091, 0.6682949662208557),
 (29234, 0.6606232523918152),
 (39219, 0.6527169942855835),
 (28966, 0.646577000617981),
 (29137, 0.6427459120750427)]

In [19]:
# microfono
r.recomendar(query_item_ids=[29028], searchOn_act_items=True, limit=0.6)[:10]

[(29028, 0.9999998807907104),
 (74821, 0.7666803002357483),
 (28923, 0.7481085658073425),
 (29005, 0.7119910717010498),
 (28968, 0.6543940305709839),
 (39161, 0.6417317390441895),
 (29289, 0.6390286087989807),
 (74754, 0.6368359327316284),
 (28958, 0.6264287233352661),
 (28970, 0.6046555638313293)]

In [18]:
# silla
r.recomendar(query_item_ids=[29143], searchOn_act_items=True, limit=0.6)[:10]

[(29143, 1.0000001192092896),
 (29348, 0.7432969808578491),
 (29064, 0.7220492362976074),
 (29129, 0.6831138730049133),
 (29347, 0.6761344075202942),
 (29018, 0.675838828086853),
 (28983, 0.6665701270103455),
 (74794, 0.6649954319000244),
 (28966, 0.6612613201141357),
 (28952, 0.6607744693756104)]

In [20]:
#r.recomendar(query_user_ids=[21724, 21726], searchOn_act_users=True, limit=0.8)

In [21]:
#r.recomendar(query_item_ids=[28863, 28871], searchOn_act_users=True, limit=0.7)

In [22]:
r.newUser_embs

tensor([[-0.2478, -0.0677,  0.0793,  ..., -0.0530, -0.0129, -0.0669],
        [-0.2478, -0.0677,  0.0793,  ..., -0.0530, -0.0129, -0.0669],
        [-0.2478, -0.0677,  0.0793,  ..., -0.0530, -0.0129, -0.0669],
        ...,
        [-0.2478, -0.0677,  0.0793,  ..., -0.0530, -0.0129, -0.0669],
        [-0.1499,  0.0059,  0.1053,  ...,  0.0096, -0.0766, -0.0581],
        [-0.1544, -0.0117,  0.1568,  ..., -0.0180,  0.0189, -0.0310]])

In [18]:
r.activeUser_embs

tensor([[ 0.3192, -0.0241, -0.2915,  ..., -0.1165,  0.1095,  0.0411],
        [ 0.3002, -0.0479,  0.0032,  ...,  0.0162,  0.1553, -0.0342],
        [ 0.4004, -0.0542, -0.3557,  ...,  0.0184, -0.0594,  0.2058],
        ...,
        [ 0.2329, -0.1110, -0.2166,  ..., -0.0215,  0.4049,  0.1638],
        [ 0.3030, -0.2221, -0.2273,  ..., -0.0230,  0.1322,  0.1733],
        [ 0.2253, -0.0509, -0.3024,  ...,  0.0155, -0.0522,  0.0342]])

In [19]:
r.activeItem_embs

tensor([[ 0.2910, -0.0460, -0.2071,  ...,  0.0156,  0.7747,  0.0952],
        [ 0.2818, -0.0875, -0.2786,  ..., -0.1080,  0.1367,  0.4187],
        [ 0.2780, -0.0760, -0.2691,  ...,  0.0087,  0.1218,  0.1433],
        ...,
        [ 0.3849, -0.0944, -0.1464,  ...,  0.0619,  0.1363,  0.2019],
        [ 0.0826, -0.0628, -0.2057,  ...,  0.0445,  0.0963,  0.0569],
        [ 0.1099,  0.1584, -0.2032,  ...,  0.0713,  0.0286, -0.1603]])